In [1]:
# This is a copy of my private kaggle kernel: https://www.kaggle.com/joatom/main-features
# fork of https://www.kaggle.com/inversion/atomic-distance-benchmark/

#J-Coupling:https://www.youtube.com/watch?v=vnkk3eli1Hc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from tqdm import tqdm_notebook as tqdm

import gc
import psutil
import os
import time
from functools import reduce
from sklearn.preprocessing import LabelEncoder

print(os.listdir("../input"))

['structures', 'mul-charges', 'champs-scalar-coupling']


# General Functions

In [2]:
in_path='../input/champs-scalar-coupling/'
struct_path= '../input/mul-charges/' #'../input/structures/'

In [3]:
# https://www.kaggle.com/todnewman/keras-neural-net-for-champs/comments
def show_ram_usage():
    py = psutil.Process(os.getpid())
    print('RAM usage: {} GB'.format(py.memory_info()[0]/2. ** 30))

In [4]:
# https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# https://www.kaggle.com/c/champs-scalar-coupling/discussion/96655#latest-565815
# guiferviz comment

def reduce_memory_usage(df, deep=True, verbose=True, categories=True):
    # All types that we want to change for "lighter" ones.
    # int8 and float16 are not include because we cannot reduce
    # those data types.
    # float32 is not include because float16 has too low precision.
    numeric2reduce = ["int16", "int32", "int64", "float64"]
    start_mem = 0
    if verbose:
        start_mem = df.memory_usage().sum() / 1024**2

    for col, col_type in df.dtypes.iteritems():
        best_type = None
        if col_type == "object":
            df[col] = df[col].astype("category")
            best_type = "category"
        elif col_type in numeric2reduce:
            downcast = "integer" if "int" in str(col_type) else "float"
            df[col] = pd.to_numeric(df[col], downcast=downcast)
            best_type = df[col].dtype.name
        # Log the conversion performed.
        if verbose and best_type is not None and best_type != str(col_type):
            print(f"Column '{col}' converted from {col_type} to {best_type}")

    if verbose:
        end_mem = df.memory_usage().sum() / 1024**2
        diff_mem = start_mem - end_mem
        percent_mem = 100 * diff_mem / start_mem
        print(f"Memory usage decreased from"
              f" {start_mem:.2f}MB to {end_mem:.2f}MB"
              f" ({diff_mem:.2f}MB, {percent_mem:.2f}% reduction)")
    return df

# Structures Data

In [5]:
structures = pd.read_pickle(struct_path+'struct_mc_ext.zip',compression='zip') #pd.read_csv(struct_path+'struct_ext.csv') 
structures=reduce_memory_usage(structures)
gc.collect()
show_ram_usage()

structures.drop(['nn_10','nn_10_dist','nn_x_10','nn_y_10','nn_z_10','nn_idx_10'],axis=1,inplace=True)

structures.head(n=10)

Column 'molecule_name' converted from object to category
Column 'atom_index' converted from int64 to int8
Column 'mulliken_charge' converted from float64 to float32
Memory usage decreased from 571.35MB to 543.60MB (27.74MB, 4.86% reduction)
RAM usage: 0.9340972900390625 GB


,molecule_name,atom_index,mulliken_charge,atom,x,y,z,num_atoms_in_mol,nn_1,nn_1_dist,...,nn_9_dist,nn_x_9,nn_y_9,nn_z_9,nn_idx_9,pca_x,pca_y,c_mol_x,c_mol_y,c_mol_z
0,dsgdb9nsd_000001,0,-0.535689,6,-0.012698,1.085804,0.008001,5,1,1.091946,...,0.0,0.012698,-1.085804,-0.008001,9999,-1.165317e-05,-1.998521e-07,-0.012689,1.085797,0.008001
1,dsgdb9nsd_000001,1,0.133921,1,0.002150,-0.006031,0.001976,5,6,1.091953,...,0.0,-0.002150,0.006031,-0.001976,9999,6.206117e-01,-2.676423e-01,-0.012689,1.085797,0.008001
2,dsgdb9nsd_000001,2,0.133922,1,1.011731,1.463751,0.000277,5,6,1.091952,...,0.0,-1.011731,-1.463751,-0.000277,9999,6.401815e-01,2.553965e-01,-0.012689,1.085797,0.008001
3,dsgdb9nsd_000001,3,0.133923,1,-0.540815,1.447527,-0.876644,5,6,1.091946,...,0.0,0.540815,-1.447527,0.876644,9999,-6.364459e-01,-8.461912e-01,-0.012689,1.085797,0.008001
4,dsgdb9nsd_000001,4,0.133923,1,-0.523814,1.437933,0.906397,5,6,1.091948,...,0.0,0.523814,-1.437933,-0.906397,9999,-6.243357e-01,8.584373e-01,-0.012689,1.085797,0.008001
5,dsgdb9nsd_000002,0,-0.707143,7,-0.040426,1.024108,0.062564,4,1,1.017187,...,0.0,0.040426,-1.024108,-0.062564,9999,4.727096e-05,-1.632185e-07,0.093086,0.934733,-0.192278
6,dsgdb9nsd_000002,1,0.235712,1,0.017257,0.012545,-0.027377,4,7,1.017190,...,0.0,-0.017257,-0.012545,0.027377,9999,-4.756682e-01,-8.043813e-01,0.093086,0.934733,-0.192278
7,dsgdb9nsd_000002,2,0.235712,1,0.915789,1.358745,-0.028758,4,7,1.017187,...,0.0,-0.915789,-1.358745,0.028758,9999,-4.589103e-01,8.140550e-01,0.093086,0.934733,-0.192278
8,dsgdb9nsd_000002,3,0.235720,1,-0.520278,1.343532,-0.775543,4,7,1.017208,...,0.0,0.520278,-1.343532,0.775543,9999,9.345312e-01,-9.673544e-03,0.093086,0.934733,-0.192278
9,dsgdb9nsd_000003,0,-0.589706,8,-0.034360,0.977540,0.007602,3,1,0.962107,...,0.0,0.034360,-0.977540,-0.007602,9999,-1.738976e-07,3.961405e-01,0.300732,0.766301,0.003276


## Count number of atoms per molecule

In [6]:
atoms_in_molecule = structures.groupby(['molecule_name','atom']).count()[['x']].unstack(fill_value=0).fillna(0)
atoms_in_molecule.columns=atoms_in_molecule.columns.droplevel()

atoms_in_molecule.rename({1:'H', 6:'C', 7:'N', 8:'O', 9:'F'}, axis=1, inplace=True) 

atoms_in_molecule=reduce_memory_usage(atoms_in_molecule)
gc.collect()
show_ram_usage()

atoms_in_molecule.head()

Column 'H' converted from float64 to float32
Column 'C' converted from float64 to float32
Column 'N' converted from float64 to float32
Column 'O' converted from float64 to float32
Column 'F' converted from float64 to float32
Memory usage decreased from 11.49MB to 8.99MB (2.49MB, 21.72% reduction)
RAM usage: 0.91888427734375 GB


atom,H,C,N,O,F
molecule_name,,,,,
dsgdb9nsd_000001,4.0,1.0,0.0,0.0,0.0
dsgdb9nsd_000002,3.0,0.0,1.0,0.0,0.0
dsgdb9nsd_000003,2.0,0.0,0.0,1.0,0.0
dsgdb9nsd_000004,2.0,2.0,0.0,0.0,0.0
dsgdb9nsd_000005,1.0,1.0,1.0,0.0,0.0


# Train and Test Data

In [7]:
train = pd.read_csv(in_path+'train.csv')#, index_col='id')
test = pd.read_csv(in_path+'test.csv')#, index_col='id')

In [8]:
display(test.head())

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [9]:
%%time
# Map the atom structure data into train and test files

def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left', #right
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'], copy=False)
    
    
    df.drop('atom_index', axis=1, inplace=True)
    df.rename(columns={'atom': f'atom_{atom_idx}',
                            'atom_index' : f'nn_idx_0_{atom_idx}',
                            #'x_norm_std': f'x_norm_std_{atom_idx}',
                            #'y_norm_std': f'y_norm_std_{atom_idx}',
                            #'z_norm_std': f'z_norm_std_{atom_idx}',
                            #'x_norm_minmax': f'x_norm_minmax_{atom_idx}',
                            #'y_norm_minmax': f'y_norm_minmax_{atom_idx}',
                            #'z_norm_minmax': f'z_norm_minmax_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}',
                            'nn_1': f'nn1_{atom_idx}',
                            'nn_2': f'nn2_{atom_idx}',
                            'nn_3': f'nn3_{atom_idx}',
                            'nn_4': f'nn4_{atom_idx}',
                            'nn_5': f'nn5_{atom_idx}',
                            'nn_6': f'nn6_{atom_idx}',
                            'nn_7': f'nn7_{atom_idx}',
                            'nn_8': f'nn8_{atom_idx}',
                            'nn_9': f'nn9_{atom_idx}',
                            #'nn_10': f'nn10_{atom_idx}',
                            'nn_1_dist': f'nn1_dist_{atom_idx}',
                            'nn_2_dist': f'nn2_dist_{atom_idx}',
                            'nn_3_dist': f'nn3_dist_{atom_idx}',
                            'nn_4_dist': f'nn4_dist_{atom_idx}',
                            'nn_5_dist': f'nn5_dist_{atom_idx}',
                            'nn_6_dist': f'nn6_dist_{atom_idx}',
                            'nn_7_dist': f'nn7_dist_{atom_idx}',
                            'nn_8_dist': f'nn8_dist_{atom_idx}',
                            'nn_9_dist': f'nn9_dist_{atom_idx}',
                            #'nn_10_dist': f'nn10_dist_{atom_idx}',
                            'nn_x_1': f'nn_x_1_{atom_idx}',
                            'nn_x_2': f'nn_x_2_{atom_idx}',
                            'nn_x_3': f'nn_x_3_{atom_idx}',
                            'nn_x_4': f'nn_x_4_{atom_idx}',
                            'nn_x_5': f'nn_x_5_{atom_idx}',
                            'nn_x_6': f'nn_x_6_{atom_idx}',
                            'nn_x_7': f'nn_x_7_{atom_idx}',
                            'nn_x_8': f'nn_x_8_{atom_idx}',
                            'nn_x_9': f'nn_x_9_{atom_idx}',
                            #'nn_x_10': f'nn_x_10_{atom_idx}',
                            'nn_y_1': f'nn_y_1_{atom_idx}',
                            'nn_y_2': f'nn_y_2_{atom_idx}',
                            'nn_y_3': f'nn_y_3_{atom_idx}',
                            'nn_y_4': f'nn_y_4_{atom_idx}',
                            'nn_y_5': f'nn_y_5_{atom_idx}',
                            'nn_y_6': f'nn_y_6_{atom_idx}',
                            'nn_y_7': f'nn_y_7_{atom_idx}',
                            'nn_y_8': f'nn_y_8_{atom_idx}',
                            'nn_y_9': f'nn_y_9_{atom_idx}',
                            #'nn_y_10': f'nn_y_10_{atom_idx}',
                            'nn_z_1': f'nn_z_1_{atom_idx}',
                            'nn_z_2': f'nn_z_2_{atom_idx}',
                            'nn_z_3': f'nn_z_3_{atom_idx}',
                            'nn_z_4': f'nn_z_4_{atom_idx}',
                            'nn_z_5': f'nn_z_5_{atom_idx}',
                            'nn_z_6': f'nn_z_6_{atom_idx}',
                            'nn_z_7': f'nn_z_7_{atom_idx}',
                            'nn_z_8': f'nn_z_8_{atom_idx}',
                            'nn_z_9': f'nn_z_9_{atom_idx}',
                            #'nn_z_10': f'nn_z_10_{atom_idx}',
                            'pca_x': f'pca_x_{atom_idx}',
                            'pca_y': f'pca_y_{atom_idx}',
                            'c_mol_x': f'c_mol_x_{atom_idx}',
                            'c_mol_y': f'c_mol_y_{atom_idx}',
                            'c_mol_z': f'c_mol_z_{atom_idx}',
                            'nn_idx_1': f'nn_idx_1_{atom_idx}',
                            'nn_idx_2': f'nn_idx_2_{atom_idx}',
                            'nn_idx_3': f'nn_idx_3_{atom_idx}',
                            'nn_idx_4': f'nn_idx_4_{atom_idx}',
                            'nn_idx_5': f'nn_idx_5_{atom_idx}',
                            'nn_idx_6': f'nn_idx_6_{atom_idx}',
                            'nn_idx_7': f'nn_idx_7_{atom_idx}',
                            'nn_idx_8': f'nn_idx_8_{atom_idx}',
                            'nn_idx_9': f'nn_idx_9_{atom_idx}',
                            'mulliken_charge': f'mulliken_charge_{atom_idx}'#,
                            #'nn_idx_10': f'nn_idx_10_{atom_idx}'
                           }, inplace=True)
    
    
    # add number of atoms
    if atom_idx==1:
        df.drop('c_mol_x_1', axis=1, inplace=True)
        df.drop('c_mol_y_1', axis=1, inplace=True)
        df.drop('c_mol_z_1', axis=1, inplace=True)
        df.drop('num_atoms_in_mol_y', axis=1, inplace=True)
        df = pd.merge(df, atoms_in_molecule, how = 'left',
                     left_on = 'molecule_name',
                     right_on = 'molecule_name', copy=False)
    
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

# 119 sec

CPU times: user 1min 23s, sys: 1min 22s, total: 2min 45s
Wall time: 2min 45s


In [10]:
del atoms_in_molecule
del structures
train=reduce_memory_usage(train)
test=reduce_memory_usage(test)
gc.collect()
show_ram_usage()

Column 'id' converted from int64 to int32
Column 'molecule_name' converted from object to category
Column 'atom_index_0' converted from int64 to int8
Column 'atom_index_1' converted from int64 to int8
Column 'type' converted from object to category
Column 'scalar_coupling_constant' converted from float64 to float32
Memory usage decreased from 2181.20MB to 2037.75MB (143.45MB, 6.58% reduction)
Column 'id' converted from int64 to int32
Column 'molecule_name' converted from object to category
Column 'atom_index_0' converted from int64 to int8
Column 'atom_index_1' converted from int64 to int8
Column 'type' converted from object to category
Column 'nn3_0' converted from int16 to int8
Column 'nn_idx_3_0' converted from int16 to int8
Column 'nn3_1' converted from int16 to int8
Column 'nn_idx_3_1' converted from int16 to int8
Memory usage decreased from 1154.11MB to 1076.86MB (77.25MB, 6.69% reduction)
RAM usage: 4.126029968261719 GB


In [11]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,mulliken_charge_0,atom_0,x_0,y_0,z_0,...,nn_y_9_1,nn_z_9_1,nn_idx_9_1,pca_x_1,pca_y_1,H,C,N,O,F
0,4658147,dsgdb9nsd_000004,2,0,2JHC,0.147765,1,-1.661639,0.0,1.0,...,0.0,-1.0,9999,-0.599540,0.0,2.0,2.0,0.0,0.0,0.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,0.147765,1,-1.661639,0.0,1.0,...,0.0,-1.0,9999,0.599540,-0.0,2.0,2.0,0.0,0.0,0.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,0.147765,1,-1.661639,0.0,1.0,...,0.0,-1.0,9999,-1.661639,0.0,2.0,2.0,0.0,0.0,0.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,0.148562,1,1.661639,0.0,1.0,...,0.0,-1.0,9999,-0.599540,0.0,2.0,2.0,0.0,0.0,0.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,0.148562,1,1.661639,0.0,1.0,...,0.0,-1.0,9999,0.599540,-0.0,2.0,2.0,0.0,0.0,0.0


In [12]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,mulliken_charge_0,atom_0,x_0,y_0,...,nn_y_9_1,nn_z_9_1,nn_idx_9_1,pca_x_1,pca_y_1,H,C,N,O,F
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807602,0.133921,1,0.002150,-0.006031,...,-1.085804,-0.008001,9999,-0.000012,-1.998521e-07,4.0,1.0,0.0,0.0,0.0
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000,0.133921,1,0.002150,-0.006031,...,-1.463751,-0.000277,9999,0.640181,2.553965e-01,4.0,1.0,0.0,0.0,0.0
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254800,0.133921,1,0.002150,-0.006031,...,-1.447527,0.876644,9999,-0.636446,-8.461912e-01,4.0,1.0,0.0,0.0,0.0
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300,0.133921,1,0.002150,-0.006031,...,-1.437933,-0.906397,9999,-0.624336,8.584373e-01,4.0,1.0,0.0,0.0,0.0
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807404,0.133922,1,1.011731,1.463751,...,-1.085804,-0.008001,9999,-0.000012,-1.998521e-07,4.0,1.0,0.0,0.0,0.0


In [13]:
k=9

In [14]:
%%time

def common_nn(df):
    k_plus=k+1 # 8 nearest neighbors + itself
    nn_cols=['atom_index_0','nn_idx_1_0','nn_idx_2_0','nn_idx_3_0','nn_idx_4_0','nn_idx_5_0','nn_idx_6_0','nn_idx_7_0','nn_idx_8_0','nn_idx_9_0']
    nn_cols_1=['atom_index_1','nn_idx_1_1','nn_idx_2_1','nn_idx_3_1','nn_idx_4_1','nn_idx_5_1','nn_idx_6_1','nn_idx_7_1','nn_idx_8_1','nn_idx_9_1']
    
    for i in range(k_plus):
        
        df['common_nn_'+str(i)]=99 #memory 99 --> int8
        if i>0:
            df['common_nn_dist_'+str(i)]=99
        
        for j in range(k_plus):
            df.loc[(df[nn_cols[i]]==df[nn_cols_1[j]]) & (df[nn_cols[i]]!=9999), 'common_nn_'+str(i)]=j
            
            # dist between atom0 and atom1 via nn-atom
            if (i>0) & (j>0):  
                df.loc[(df[nn_cols[i]]==df[nn_cols_1[j]]) & (df[nn_cols[i]]!=9999), 'common_nn_dist_'+str(i)]= df['nn'+str(i)+'_dist_0']+df['nn'+str(j)+'_dist_1']

    return df

train = common_nn(train)
test = common_nn(test)

# 55 sec

CPU times: user 42.7 s, sys: 10.5 s, total: 53.2 s
Wall time: 52.7 s


In [15]:
%%time
for i in range(k):
    train['nn_'+str(i+1)+'_0_exists'] = train['nn_idx_'+str(i+1)+'_0'] <= k
    train.drop(['nn_idx_'+str(i+1)+'_0'],axis=1,inplace=True)
    train['nn_'+str(i+1)+'_1_exists'] = train['nn_idx_'+str(i+1)+'_1'] <= k
    train.drop(['nn_idx_'+str(i+1)+'_1'],axis=1,inplace=True)
    gc.collect()
    
    test['nn_'+str(i+1)+'_0_exists'] = test['nn_idx_'+str(i+1)+'_0'] <= k
    test.drop(['nn_idx_'+str(i+1)+'_0'],axis=1,inplace=True)
    test['nn_'+str(i+1)+'_1_exists'] = test['nn_idx_'+str(i+1)+'_1'] <= k
    test.drop(['nn_idx_'+str(i+1)+'_1'],axis=1,inplace=True)
    gc.collect()

#56 s

CPU times: user 19.9 s, sys: 39.1 s, total: 59 s
Wall time: 59 s


In [16]:
train.drop(['molecule_name','atom_index_0','atom_index_1'],axis=1,inplace=True)
test.drop(['molecule_name','atom_index_0','atom_index_1'],axis=1,inplace=True)

train=reduce_memory_usage(train)
test=reduce_memory_usage(test)
gc.collect()
show_ram_usage()

Column 'common_nn_0' converted from int64 to int8
Column 'common_nn_1' converted from int64 to int8
Column 'common_nn_dist_1' converted from float64 to float32
Column 'common_nn_2' converted from int64 to int8
Column 'common_nn_dist_2' converted from float64 to float32
Column 'common_nn_3' converted from int64 to int8
Column 'common_nn_dist_3' converted from float64 to float32
Column 'common_nn_4' converted from int64 to int8
Column 'common_nn_dist_4' converted from float64 to float32
Column 'common_nn_5' converted from int64 to int8
Column 'common_nn_dist_5' converted from float64 to float32
Column 'common_nn_6' converted from int64 to int8
Column 'common_nn_dist_6' converted from float64 to float32
Column 'common_nn_7' converted from int64 to int8
Column 'common_nn_dist_7' converted from float64 to float32
Column 'common_nn_8' converted from int64 to int8
Column 'common_nn_dist_8' converted from float64 to float32
Column 'common_nn_9' converted from int64 to int8
Column 'common_nn_di

# Angles

## 2J
find the one connecting atom

In [17]:
test.head(n=15)

,id,type,mulliken_charge_0,atom_0,x_0,y_0,z_0,num_atoms_in_mol_x,nn1_0,nn1_dist_0,...,nn_5_0_exists,nn_5_1_exists,nn_6_0_exists,nn_6_1_exists,nn_7_0_exists,nn_7_1_exists,nn_8_0_exists,nn_8_1_exists,nn_9_0_exists,nn_9_1_exists
0,4658147,2JHC,0.147765,1,-1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,False,False
1,4658148,1JHC,0.147765,1,-1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,False,False
2,4658149,3JHH,0.147765,1,-1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,False,False
3,4658150,1JHC,0.148562,1,1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,False,False
4,4658151,2JHC,0.148562,1,1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,False,False
5,4658152,1JHC,0.080328,1,1.005284,1.810158,0.004656,9,6,1.102328,...,True,True,True,True,True,True,True,True,False,False
6,4658153,3JHC,0.080328,1,1.005284,1.810158,0.004656,9,6,1.102328,...,True,True,True,True,True,True,True,True,False,False
7,4658154,2JHH,0.080328,1,1.005284,1.810158,0.004656,9,6,1.102328,...,True,True,True,True,True,True,True,True,False,False
8,4658155,2JHH,0.080328,1,1.005284,1.810158,0.004656,9,6,1.102328,...,True,True,True,True,True,True,True,True,False,False
9,4658156,1JHC,0.099221,1,-0.546896,1.793435,-0.872510,9,6,1.102327,...,True,True,True,True,True,True,True,True,False,False


Find nearest neighbor with min distance to atom1 and atom2

In [18]:
test.head(n=15)[['common_nn_dist_'+str(i+1) for i in range(k)]].idxmin(axis=1).apply(lambda x: (str(x))[str(x).find('_',-1):])

0     1
1     2
2     1
3     2
4     1
5     3
6     1
7     1
8     1
9     3
10    1
11    1
12    2
13    4
14    1
dtype: object

In [19]:
%%time
train['2J_nn']=pd.to_numeric(train[['common_nn_dist_'+str(i+1) for i in range(k)]].idxmin(axis=1).apply(lambda x: (str(x))[str(x).find('_',-1):]),downcast='integer')
test['2J_nn']=pd.to_numeric(test[['common_nn_dist_'+str(i+1) for i in range(k)]].idxmin(axis=1).apply(lambda x: (str(x))[str(x).find('_',-1):]),downcast='integer')
#22sec

CPU times: user 17.6 s, sys: 3.59 s, total: 21.2 s
Wall time: 21.3 s


In [20]:
gc.collect()
show_ram_usage()

RAM usage: 4.572566986083984 GB


## 3J
Try to find 2nd atom to establish 3 bond connection. Choose the one with 2nd shortest distance between atom0 and atom1. That's just an aproximation, but should be sufficient calculate a dihedral angel.

In [21]:
test.head(n=15)[['common_nn_dist_'+str(i+1) for i in range(k)]]

,common_nn_dist_1,common_nn_dist_2,common_nn_dist_3,common_nn_dist_4,common_nn_dist_5,common_nn_dist_6,common_nn_dist_7,common_nn_dist_8,common_nn_dist_9
0,2.261178,99.000000,4.385376,99.000000,99.000000,99.000000,99.000000,99.000000,99.0
1,99.000000,3.460257,5.584455,99.000000,99.000000,99.000000,99.000000,99.000000,99.0
2,3.323277,3.323277,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.0
3,99.000000,3.460257,5.584455,99.000000,99.000000,99.000000,99.000000,99.000000,99.0
4,2.261178,99.000000,4.385376,99.000000,99.000000,99.000000,99.000000,99.000000,99.0
5,99.000000,2.885292,2.877869,3.490865,5.101932,4.977977,5.680575,6.887329,99.0
6,3.439981,4.422874,5.047583,3.490865,3.564297,99.000000,4.142633,4.717614,99.0
7,2.204655,99.000000,3.570017,4.169564,5.501122,5.280234,5.502154,7.249436,99.0
8,2.195180,3.567965,99.000000,4.108404,6.086709,5.902890,6.664999,7.667284,99.0
9,99.000000,2.885292,2.877852,3.490866,5.102118,4.977563,5.679115,6.887241,99.0


In [22]:
test.head(n=15)[['common_nn_dist_'+str(i+1) for i in range(k)]].rank(method='first',axis=1).apply(lambda x: x.where(x>1)).idxmin(axis=1)

0     common_nn_dist_3
1     common_nn_dist_3
2     common_nn_dist_2
3     common_nn_dist_3
4     common_nn_dist_3
5     common_nn_dist_2
6     common_nn_dist_4
7     common_nn_dist_3
8     common_nn_dist_2
9     common_nn_dist_2
10    common_nn_dist_4
11    common_nn_dist_2
12    common_nn_dist_3
13    common_nn_dist_1
14    common_nn_dist_4
dtype: object

In [23]:
%%time
train['3J_nn']=pd.to_numeric(train[['common_nn_dist_'+str(i+1) for i in range(k)]].rank(method='first',axis=1).apply(lambda x: x.where(x>1)).idxmin(axis=1).apply(lambda x: (str(x))[str(x).find('_',-1):]),downcast='integer')
test['3J_nn']=pd.to_numeric(test[['common_nn_dist_'+str(i+1) for i in range(k)]].rank(method='first',axis=1).apply(lambda x: x.where(x>1)).idxmin(axis=1).apply(lambda x: (str(x))[str(x).find('_',-1):]),downcast='integer')
#22sec

CPU times: user 22.6 s, sys: 2.98 s, total: 25.6 s
Wall time: 25 s


In [24]:
# https://www.kaggle.com/soerendip/calculate-angles-and-dihedrals-with-networkx
'''
def cosinus(x0, x1, x2):
    e0 = (x0-x1)
    e1 = (x2-x1)
    e0 = (e0 / np.linalg.norm(e0))
    e1 = (e1 / np.linalg.norm(e1))
    cosinus = np.dot(e0, e1)
    return np.round(cosinus, 5)
'''

def dihedral(x0, x1, x2, x3):
    #print(x0, x1, x2, x3)
    #x0=np.array(x0, dtype=np.float)#np.round(x0,5)
    #x1=np.array(x1, dtype=np.float)
    #x2=np.array(x2, dtype=np.float)
    #x3=np.array(x3, dtype=np.float)
    
    b0 = -1.0 * (x1 - x0)
    b1 = x2 - x1
    b2 = x3 - x2

    #https://www.kaggle.com/ggeo79/j-coupling-lightbgm-gpu-dihedral-angle/notebook
    #b1 /= np.linalg.norm(b1)
    
    b0xb1 = np.cross(b0, b1,axis=1)
    b1xb2 = np.cross(b2, b1,axis=1)

    b0xb1_x_b1xb2 = np.cross(b0xb1, b1xb2,axis=1)
    
    #print(b0xb1_x_b1xb2.shape, b1.shape)
    
    y = np.sum(b0xb1_x_b1xb2 * b1, axis=1) * (1.0/np.linalg.norm(b1,axis=1))
    x = np.sum(b0xb1 * b1xb2, axis=1)
    
    
    #v = b0 - np.dot(b0, b1) * b1
    #w = b2 - np.dot(b2, b1) * b1

    #v /= np.linalg.norm(v)
    #w /= np.linalg.norm(w)
    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    #x = np.dot(v, w)
    #y = np.dot(np.cross(b1, v), w)
        
    grad = np.arctan2(y, x)
    
    return grad #np.degrees(np.arctan2(y, x)) #grad


In [25]:
gc.collect()
show_ram_usage()
test.head(n=15)

RAM usage: 4.614585876464844 GB


,id,type,mulliken_charge_0,atom_0,x_0,y_0,z_0,num_atoms_in_mol_x,nn1_0,nn1_dist_0,...,nn_6_0_exists,nn_6_1_exists,nn_7_0_exists,nn_7_1_exists,nn_8_0_exists,nn_8_1_exists,nn_9_0_exists,nn_9_1_exists,2J_nn,3J_nn
0,4658147,2JHC,0.147765,1,-1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,1,3
1,4658148,1JHC,0.147765,1,-1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,2,3
2,4658149,3JHH,0.147765,1,-1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,1,2
3,4658150,1JHC,0.148562,1,1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,2,3
4,4658151,2JHC,0.148562,1,1.661639,0.000000,1.000000,4,6,1.062099,...,False,False,False,False,False,False,False,False,1,3
5,4658152,1JHC,0.080328,1,1.005284,1.810158,0.004656,9,6,1.102328,...,True,True,True,True,True,True,False,False,3,2
6,4658153,3JHC,0.080328,1,1.005284,1.810158,0.004656,9,6,1.102328,...,True,True,True,True,True,True,False,False,1,4
7,4658154,2JHH,0.080328,1,1.005284,1.810158,0.004656,9,6,1.102328,...,True,True,True,True,True,True,False,False,1,3
8,4658155,2JHH,0.080328,1,1.005284,1.810158,0.004656,9,6,1.102328,...,True,True,True,True,True,True,False,False,1,2
9,4658156,1JHC,0.099221,1,-0.546896,1.793435,-0.872510,9,6,1.102327,...,True,True,True,True,True,True,False,False,3,2


In [26]:
#%%time
#train.head(n=40).apply(lambda x: dihedral(x[['x_0','y_0','z_0']].values,x[['nn_x_'+str(x['2J_nn'])+'_0','nn_y_'+str(x['2J_nn'])+'_0','nn_z_'+str(x['2J_nn'])+'_0']].values,x[['nn_x_'+str(x['3J_nn'])+'_0','nn_y_'+str(x['3J_nn'])+'_0','nn_z_'+str(x['3J_nn'])+'_0']].values,x[['x_1','y_1','z_1']].values),axis=1)

# Distance and additional features

In [27]:

def cos_dist(df_p_0, df_p_1):
    a = 1.0 - np.average(df_p_0 * df_p_1, axis=1)/np.sqrt(np.average(np.square(df_p_0),axis=1)*np.average(np.square(df_p_1),axis=1))
    return np.where(np.isnan(a), 0, a)

def distance_features(df, norm, euclidean, cos, dist_center,nn_cos_dist):
    # https://www.kaggle.com/seriousran/just-speed-up-calculate-distance-from-benchmark
    df_p_0 = df[['x' + norm + '_0', 'y' + norm + '_0', 'z' + norm + '_0']].values
    df_p_1 = df[['x' + norm + '_1', 'y' + norm + '_1', 'z' + norm + '_1']].values
      
    if euclidean:
        # Euclidean
        print('euclidean')
        df['dist'+norm] = np.linalg.norm(df_p_0 - df_p_1, axis=1)
    
    if cos:
        print('cos')
        #https://www.kaggle.com/urmaspitsi/speed-up-distance-calc-even-more
        #https://github.com/scipy/scipy/blob/master/scipy/spatial/distance.py
        df['dist_cos'+norm] = cos_dist(df_p_0, df_p_1)
 
    if dist_center:
        print('dist_center')
        df_c_0 = df[['c_mol_x_0', 'c_mol_y_0', 'c_mol_z_0']].values
        # Euclidean
        df['dist_mol_c_0'+norm] = np.linalg.norm(df_p_0 - df_c_0, axis=1)
        df['dist_mol_c_1'+norm] = np.linalg.norm(df_p_1 - df_c_0, axis=1)

        
    if nn_cos_dist:
    
        print('nn_cos_dist')
        df['2J_angle']=9999
        df['3J_angle']=9999

        nn=k

        for n in range(1, nn+1):
            # df_p_0 and df_p_1 so that nn_ is centered (0,0,0)
            df_p_nn_0 = df[['nn_x_'+str(n) + '_0', 'nn_y_'+str(n) + '_0', 'nn_z_'+str(n) + '_0']].values
            df_p_nn_1 = df[['nn_x_'+str(n) + '_1', 'nn_y_'+str(n) + '_1', 'nn_z_'+str(n) + '_1']].values
            # shift nn_x_ to original position (df_p_nn_0+df_p_0)
            # move atom_0 and atom_1
            df_shift_0_by_nn_0 = df_p_0 + (df_p_nn_0+df_p_0)*(-1)
            df_shift_1_by_nn_0 = df_p_1 + (df_p_nn_0+df_p_0)*(-1)
            df_shift_0_by_nn_1 = df_p_0 + (df_p_nn_1+df_p_1)*(-1)
            df_shift_1_by_nn_1 = df_p_1 + (df_p_nn_1+df_p_1)*(-1)

            df['dist_nn_0_'+str(n)+norm] = cos_dist(df_shift_0_by_nn_0, df_shift_1_by_nn_0)
            df['dist_nn_1_'+str(n)+norm] = cos_dist(df_shift_0_by_nn_1, df_shift_1_by_nn_1)

            # angle for 2J-connection
            df.loc[(df['2J_nn']==n), '2J_angle'] = df['dist_nn_0_'+str(n)+norm]
            df.loc[(df['3J_nn']==n), '3J_angle'] = df['dist_nn_0_'+str(n)+norm]
            # xyz for dihedral calculation
            df.loc[(df['2J_nn']==n), '2J_x'] = df['nn_x_'+str(n)+norm+'_0']
            df.loc[(df['3J_nn']==n), '3J_x'] = df['nn_x_'+str(n)+norm+'_0']
            df.loc[(df['2J_nn']==n), '2J_y'] = df['nn_y_'+str(n)+norm+'_0']
            df.loc[(df['3J_nn']==n), '3J_y'] = df['nn_y_'+str(n)+norm+'_0']
            df.loc[(df['2J_nn']==n), '2J_z'] = df['nn_z_'+str(n)+norm+'_0']
            df.loc[(df['3J_nn']==n), '3J_z'] = df['nn_z_'+str(n)+norm+'_0']
            

        # dihedral
        df_p_2J = df[['2J_x', '2J_y', '2J_z']].values
        df_p_3J = df[['3J_x', '3J_y', '3J_z']].values
        df['dihedral']=dihedral(df_p_0,df_p_2J,df_p_3J, df_p_1)
    return df


In [28]:
%%time

#train = distance_features(train, '', True, True, True, True, False)

print('Start train')
train = distance_features(train, '', True, True, True,True)

#train = distance_features(train, '_norm_std', True, True, False, False, False)
#train = distance_features(train, '_norm_minmax', True, False, False, False, True)

#test = distance_features(test, '', True, True, True, True, False)

print('Start test')
test = distance_features(test, '', True, True, True, True)

#test = distance_features(test, '', True, True, False, True, True)


#test = distance_features(test, '_norm_std', True, True, False, False, False)
#test = distance_features(test, '_norm_minmax', True, False, False, False, True)

# 1min

Start train
euclidean
cos
dist_center
nn_cos_dist


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


Start test
euclidean
cos
dist_center
nn_cos_dist
CPU times: user 35.9 s, sys: 46.9 s, total: 1min 22s
Wall time: 1min 22s


In [29]:
train.head(10)

,id,type,scalar_coupling_constant,mulliken_charge_0,atom_0,x_0,y_0,z_0,num_atoms_in_mol_x,nn1_0,...,dist_nn_1_5,dist_nn_0_6,dist_nn_1_6,dist_nn_0_7,dist_nn_1_7,dist_nn_0_8,dist_nn_1_8,dist_nn_0_9,dist_nn_1_9,dihedral
0,0,1JHC,84.807602,0.133921,1,0.002150,-0.006031,0.001976,5,6,...,1.901529,1.901529,1.901529,1.901529,1.901529,1.901529,1.901529,1.901529,1.901529,0.609247
1,1,2JHH,-11.257000,0.133921,1,0.002150,-0.006031,0.001976,5,6,...,1.557883,1.557883,1.557883,1.557883,1.557883,1.557883,1.557883,1.557883,1.557883,2.085292
2,2,2JHH,-11.254800,0.133921,1,0.002150,-0.006031,0.001976,5,6,...,1.976519,1.976519,1.976519,1.976519,1.976519,1.976519,1.976519,1.976519,1.976519,2.091930
3,3,2JHH,-11.254300,0.133921,1,0.002150,-0.006031,0.001976,5,6,...,1.671855,1.671855,1.671855,1.671855,1.671855,1.671855,1.671855,1.671855,1.671855,-2.086060
4,4,1JHC,84.807404,0.133922,1,1.011731,1.463751,0.000277,5,6,...,0.184104,0.184104,0.184104,0.184104,0.184104,0.184104,0.184104,0.184104,0.184104,0.174437
5,5,2JHH,-11.254100,0.133922,1,1.011731,1.463751,0.000277,5,6,...,0.502911,0.502911,0.502911,0.502911,0.502911,0.502911,0.502911,0.502911,0.502911,-1.045449
6,6,2JHH,-11.254800,0.133922,1,1.011731,1.463751,0.000277,5,6,...,0.502329,0.502329,0.502329,0.502329,0.502329,0.502329,0.502329,0.502329,0.502329,1.045446
7,7,1JHC,84.809303,0.133923,1,-0.540815,1.447527,-0.876644,5,6,...,0.185382,0.185382,0.185382,0.185382,0.185382,0.185382,0.185382,0.185382,0.185382,0.175366
8,8,2JHH,-11.254300,0.133923,1,-0.540815,1.447527,-0.876644,5,6,...,0.503111,0.503111,0.503111,0.503111,0.503111,0.503111,0.503111,0.503111,0.503111,-1.049550
9,9,1JHC,84.809502,0.133923,1,-0.523814,1.437933,0.906397,5,6,...,0.184437,0.184437,0.184437,0.184437,0.184437,0.184437,0.184437,0.184437,0.184437,-0.099739


In [30]:
gc.collect()

280

In [31]:
train=reduce_memory_usage(train)
test=reduce_memory_usage(test)
gc.collect()
show_ram_usage()

Column '2J_angle' converted from float64 to float32
Column '3J_angle' converted from float64 to float32
Column '2J_x' converted from float64 to float32
Column '3J_x' converted from float64 to float32
Column '2J_y' converted from float64 to float32
Column '3J_y' converted from float64 to float32
Column '2J_z' converted from float64 to float32
Column '3J_z' converted from float64 to float32
Column 'dihedral' converted from float64 to float32
Memory usage decreased from 3029.76MB to 2869.84MB (159.92MB, 5.28% reduction)
Column '2J_angle' converted from float64 to float32
Column '3J_angle' converted from float64 to float32
Column '2J_x' converted from float64 to float32
Column '3J_x' converted from float64 to float32
Column '2J_y' converted from float64 to float32
Column '3J_y' converted from float64 to float32
Column '2J_z' converted from float64 to float32
Column '3J_z' converted from float64 to float32
Column 'dihedral' converted from float64 to float32
Memory usage decreased from 1609.

In [32]:
'''
temp_tt=pd.concat([train[test.columns[:]], test], axis =0)[['type','dist']] .groupby('type').agg(['mean', 'std'])
temp_tt.columns=temp_tt.columns.droplevel()
temp_tt.columns=['type_dist_mean','type_dist_std']
temp_tt.head()'''

"\ntemp_tt=pd.concat([train[test.columns[:]], test], axis =0)[['type','dist']] .groupby('type').agg(['mean', 'std'])\ntemp_tt.columns=temp_tt.columns.droplevel()\ntemp_tt.columns=['type_dist_mean','type_dist_std']\ntemp_tt.head()"

In [33]:
def additional_features(df):
    df['bonds'] = pd.to_numeric(df['type'].str[0:1])
    
    # center of both compared atoms
    df['c_x'] = (df['x_0']+df['x_1']) / 2
    df['c_y'] = (df['y_0']+df['y_1']) / 2
    df['c_z'] = (df['z_0']+df['z_1']) / 2
    
    '''
    # mean and std dist per type
    df = pd.merge(df, temp_tt, how = 'left',
                     left_on = 'type',
                     right_on = 'type', copy=False)
    df['dist_mean_per_type'] = df['dist']-df['type_dist_mean']
    df['dist_std_per_type'] = df['dist']-df['type_dist_std']
    #df['dist_mean_per_type'] = df_both.groupby('type')['dist'].transform(np.mean)-df['dist']
    #df['dist_std_per_type'] = df_both.groupby('type')['dist'].transform(np.std)-df['dist']
    '''
    
    return df


In [34]:
%%time

train = additional_features(train)
test = additional_features(test)

temp_tt=None

train=reduce_memory_usage(train)
test=reduce_memory_usage(test)
gc.collect()
show_ram_usage()


Column 'bonds' converted from int64 to int8
Memory usage decreased from 2958.68MB to 2927.59MB (31.10MB, 1.05% reduction)
Column 'bonds' converted from int64 to int8
Memory usage decreased from 1571.03MB to 1554.30MB (16.73MB, 1.06% reduction)
RAM usage: 6.303367614746094 GB
CPU times: user 4.61 s, sys: 500 ms, total: 5.11 s
Wall time: 5.03 s


In [35]:
%%time

for c in ['atom_0','atom_1']:
    lbl = LabelEncoder()
    lbl.fit(list(train[c].values) + list(test[c].values))
    train[c] = lbl.transform(list(train[c].values))
    test[c] = lbl.transform(list(test[c].values))

CPU times: user 8.14 s, sys: 648 ms, total: 8.79 s
Wall time: 8.8 s


In [36]:

train=reduce_memory_usage(train)
test=reduce_memory_usage(test)
gc.collect()
show_ram_usage()

Column 'atom_0' converted from int64 to int8
Column 'atom_1' converted from int64 to int8
Memory usage decreased from 2989.78MB to 2927.59MB (62.19MB, 2.08% reduction)
Column 'atom_0' converted from int64 to int8
Column 'atom_1' converted from int64 to int8
Memory usage decreased from 1587.76MB to 1554.30MB (33.45MB, 2.11% reduction)
RAM usage: 6.365875244140625 GB


mark nearest neighbor as available (1) or missing (0)

In [37]:
test.head(n=10)

,id,type,mulliken_charge_0,atom_0,x_0,y_0,z_0,num_atoms_in_mol_x,nn1_0,nn1_dist_0,...,dist_nn_1_7,dist_nn_0_8,dist_nn_1_8,dist_nn_0_9,dist_nn_1_9,dihedral,bonds,c_x,c_y,c_z
0,4658147,2JHC,0.147765,0,-1.661639,0.000000,1.000000,4,6,1.062099,...,0.998327,0.998327,0.998327,0.998327,0.998327,0.000000,2,-0.531050,0.000000,1.000000
1,4658148,1JHC,0.147765,0,-1.661639,0.000000,1.000000,4,6,1.062099,...,0.117179,0.117179,0.117179,0.117179,0.117179,0.000000,1,-1.130589,0.000000,1.000000
2,4658149,3JHH,0.147765,0,-1.661639,0.000000,1.000000,4,6,1.062099,...,1.468233,1.468233,1.468233,1.468233,1.468233,0.000000,3,0.000000,0.000000,1.000000
3,4658150,1JHC,0.148562,0,1.661639,0.000000,1.000000,4,6,1.062099,...,0.117179,0.117179,0.117179,0.117179,0.117179,0.000000,1,1.130589,0.000000,1.000000
4,4658151,2JHC,0.148562,0,1.661639,0.000000,1.000000,4,6,1.062099,...,0.998327,0.998327,0.998327,0.998327,0.998327,0.000000,2,0.531050,0.000000,1.000000
5,4658152,1JHC,0.080328,0,1.005284,1.810158,0.004656,9,6,1.102328,...,0.065720,0.045829,0.045829,0.130986,0.130986,-0.125959,1,0.495231,1.601285,0.005164
6,4658153,3JHC,0.080328,0,1.005284,1.810158,0.004656,9,6,1.102328,...,0.576477,0.070777,0.411090,1.126279,1.126279,-0.601175,3,0.821616,0.628431,-0.554463
7,4658154,2JHH,0.080328,0,1.005284,1.810158,0.004656,9,6,1.102328,...,0.190170,0.120978,0.120978,0.371197,0.371197,0.636895,2,0.229194,1.801797,-0.433927
8,4658155,2JHH,0.080328,0,1.005284,1.810158,0.004656,9,6,1.102328,...,0.102771,0.102768,0.102768,0.380641,0.380641,-0.635462,2,0.237627,1.766539,0.457837
9,4658156,1JHC,0.099221,0,-0.546896,1.793435,-0.872510,9,6,1.102327,...,0.091023,0.045832,0.045832,0.131744,0.131744,0.126659,1,-0.280859,1.592924,-0.433420


In [38]:
train.head(n=15)

,id,type,scalar_coupling_constant,mulliken_charge_0,atom_0,x_0,y_0,z_0,num_atoms_in_mol_x,nn1_0,...,dist_nn_1_7,dist_nn_0_8,dist_nn_1_8,dist_nn_0_9,dist_nn_1_9,dihedral,bonds,c_x,c_y,c_z
0,0,1JHC,84.807602,0.133921,0,0.002150,-0.006031,0.001976,5,6,...,1.901529,1.901529,1.901529,1.901529,1.901529,0.609247,1,-0.005274,0.539886,0.004989
1,1,2JHH,-11.257000,0.133921,0,0.002150,-0.006031,0.001976,5,6,...,1.557883,1.557883,1.557883,1.557883,1.557883,2.085292,2,0.506941,0.728860,0.001126
2,2,2JHH,-11.254800,0.133921,0,0.002150,-0.006031,0.001976,5,6,...,1.976519,1.976519,1.976519,1.976519,1.976519,2.091930,2,-0.269332,0.720748,-0.437334
3,3,2JHH,-11.254300,0.133921,0,0.002150,-0.006031,0.001976,5,6,...,1.671855,1.671855,1.671855,1.671855,1.671855,-2.086060,2,-0.260832,0.715951,0.454187
4,4,1JHC,84.807404,0.133922,0,1.011731,1.463751,0.000277,5,6,...,0.184104,0.184104,0.184104,0.184104,0.184104,0.174437,1,0.499516,1.274778,0.004139
5,5,2JHH,-11.254100,0.133922,0,1.011731,1.463751,0.000277,5,6,...,0.502911,0.502911,0.502911,0.502911,0.502911,-1.045449,2,0.235458,1.455639,-0.438184
6,6,2JHH,-11.254800,0.133922,0,1.011731,1.463751,0.000277,5,6,...,0.502329,0.502329,0.502329,0.502329,0.502329,1.045446,2,0.243959,1.450842,0.453337
7,7,1JHC,84.809303,0.133923,0,-0.540815,1.447527,-0.876644,5,6,...,0.185382,0.185382,0.185382,0.185382,0.185382,0.175366,1,-0.276757,1.266665,-0.434321
8,8,2JHH,-11.254300,0.133923,0,-0.540815,1.447527,-0.876644,5,6,...,0.503111,0.503111,0.503111,0.503111,0.503111,-1.049550,2,-0.532314,1.442730,0.014877
9,9,1JHC,84.809502,0.133923,0,-0.523814,1.437933,0.906397,5,6,...,0.184437,0.184437,0.184437,0.184437,0.184437,-0.099739,1,-0.268256,1.261868,0.457199


In [39]:
#train.info(max_cols=200)

In [40]:
#train.to_csv('temp_train.csv', index=False)
#test.to_csv('temp_test.csv', index=False)

for i in train['type'].unique():
    print(i)
    train[train['type']==i].to_pickle('train_ext_'+i+'.zip', compression = 'zip') 
    test[test['type']==i].to_pickle('test_ext_'+i+'.zip', compression = 'zip')


1JHC
2JHH
1JHN
2JHN
2JHC
3JHH
3JHC
3JHN
